In [86]:
%%html
<style> table {float:left} </style>

In [2]:
!pip install torch tqdm lazyme nltk gensim
!python -m nltk.downloader punkt

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/2.7/bin/pip", line 10, in <module>
    sys.exit(main())
TypeError: 'module' object is not callable
/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: No module named nltk


In [1]:
import numpy as np
from tqdm import tqdm

import pandas as pd

from gensim.corpora import Dictionary

import torch
from torch import nn, optim, tensor, autograd
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
try: # Use the default NLTK tokenizer.
    from nltk import word_tokenize, sent_tokenize 
    # Testing whether it works. 
    # Sometimes it doesn't work on some machines because of setup issues.
    word_tokenize(sent_tokenize("This is a foobar sentence. Yes it is.")[0])
except: # Use a naive sentence tokenizer and toktok.
    import re
    from nltk.tokenize import ToktokTokenizer
    # See https://stackoverflow.com/a/25736515/610569
    sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
    # Use the toktok tokenizer that requires no dependencies.
    toktok = ToktokTokenizer()
    word_tokenize = word_tokenize = toktok.tokenize

# Classifying Toxic Comments

Lets apply what we learnt in a realistic task and **fight cyber-abuse with NLP**!

From https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/

> *The threat of abuse and harassment online means that many people stop <br>*
> *expressing themselves and give up on seeking different opinions. <br>*
> *Platforms struggle to effectively facilitate conversations, leading many <br>*
> *communities to limit or completely shut down user comments.*


The goal of the task is to build a model to detect different types of of toxicity:

 - toxic
 - severe toxic
 - threats
 - obscenity
 - insults
 - identity-based hate
 
In this part, you'll be munging the data as how I would be doing it at work. 

Your task is to train a feed-forward network on the toxic comments given the skills we have accomplished thus far.

## Digging into the data...

If you're using linux/Mac you can use these bang commands in the notebook:

```
!pip3 install kaggle
!mkdir -p /content/.kaggle/
!echo '{"username":"natgillin","key":"54ae95ab760b52c3307ed4645c6c9b5d"}' > /content/.kaggle/kaggle.json
!chmod 600 /content/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!unzip /content/.kaggle/competitions/jigsaw-toxic-comment-classification-challenge/*
```

Otherwise, download the data from https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/ 

In [3]:
!pip3 install kaggle
!mkdir -p /content/.kaggle/
!echo '{"username":"natgillin","key":"54ae95ab760b52c3307ed4645c6c9b5d"}' > /content/.kaggle/kaggle.json
!chmod 600 /content/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!unzip /content/.kaggle/competitions/jigsaw-toxic-comment-classification-challenge/*

^C
ERROR: Operation cancelled by user
mkdir: /content/.kaggle/: Read-only file system
/bin/sh: /content/.kaggle/kaggle.json: No such file or directory
chmod: /content/.kaggle/kaggle.json: No such file or directory
unzip:  cannot find or open /content/.kaggle/competitions/jigsaw-toxic-comment-classification-challenge/*, /content/.kaggle/competitions/jigsaw-toxic-comment-classification-challenge/*.zip or /content/.kaggle/competitions/jigsaw-toxic-comment-classification-challenge/*.ZIP.

No zipfiles found.


In [3]:
df_train = pd.read_csv('train.csv')
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df_train[df_train['threat'] == 1]['comment_text']

79        Hi! I am back again!\nLast warning!\nStop undo...
176       I think that your a Fagget get a oife and burn...
600       I'm also a sock puppet of this account...SUPRI...
802       Fuck you, Smith. Please have me notified when ...
1017      WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'L...
                                ...                        
157718    bitch \nyou are a fucking hore. you suck dick ...
158717    stupid head \n\nYOur dumb and you are stupid d...
158856    Hey \n\nhey faggot, are you dead yet? or are y...
159029                                  Death to Musulmans!
159400    Shalom \n\nSemite, get the fuck out of here. I...
Name: comment_text, Length: 478, dtype: object

In [5]:
df_train.iloc[3712]['comment_text']

'Please stop. If you continue to ignore our policies by introducing inappropriate pages to Wikipedia, you will be blocked.'

In [ ]:
df_train['comment_text_tokenzied'] = df_train['comment_text'].apply(word_tokenize)

In [14]:
from turicreate import SFrame
sf_train = SFrame.read_csv('train.csv')

Unable to parse line ""0000997932d777bf","Explanation"

Unable to parse line "2. Aims: Mountolive seemed to insist in claiming (erroneously) that ETA's principal aim is to create a socialist state. This is not real, has been discussed before and can be documented. The current status is ok."

Unable to parse line ""0001b41b1c6bb37e",""""

Unable to parse line "3. Secondary tactical targets: "

Unable to parse line "More"

...

Unable to parse line "2. Aims: Mountolive seemed to insist in claiming (erroneously) that ETA's principal aim is to create a socialist state. This is not real, has been discussed before and can be documented. The current status is ok."

Read 64740 lines. Lines per second: 62285.7

329896 lines failed to parse correctly

Finished parsing file /Users/liling.tan/git-stuff/tsundoku/completed/train.csv

Parsing completed. Parsed 85122 lines in 1.16068 secs.

In [ ]:
# Just in case your Jupyter kernel dies, save the tokenized text =)

# To save your tokenized text you can do this:
import pickle
with open('train_tokenized_text.pkl', 'wb') as fout:
    pickle.dump(df_train['comment_text_tokenzied'], fout)


In [7]:
# To load it back:
import pickle
with open('train_tokenized_text.pkl', 'rb') as fin:
    df_train['comment_text_tokenzied'] = pickle.load(fin)

# How to get a one-hot?

There are many variants of how to get your one-hot embeddings from the individual columns.

This is one way:

In [8]:
label_column_names = "toxic	severe_toxic	obscene	threat	insult	identity_hate".split()
df_train[label_column_names].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [9]:
torch.tensor(df_train[label_column_names].values).float()

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [10]:
# Convert one-hot to indices of the column.

print(np.argmax(df_train[label_column_names].values, axis=1))

[0 0 0 ... 0 0 0]


In [24]:
class ToxicDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.vocab = Dictionary(texts)
        special_tokens = {'<pad>': 0, '<unk>':1}
        self.vocab = Dictionary(texts)
        self.vocab.patch_with_special_tokens(special_tokens)
        
        self.vocab_size = len(self.vocab)
        
        # Vectorize labels
        self.labels = torch.tensor(labels)
        # Keep track of how many data points.
        self._len = len(texts)
        
        # Find the longest text in the data.
        self.max_len = max(len(txt) for txt in texts)
        
        self.num_labels = len(labels[0])
        
    def __getitem__(self, index):
        vectorized_sent = self.vectorize(self.texts[index])
        # To pad the sentence:
        # Pad left = 0; Pad right = max_len - len of sent.
        pad_dim = (0, self.max_len - len(vectorized_sent))
        vectorized_sent_padded = F.pad(vectorized_sent, pad_dim, 'constant')
        return {'x':vectorized_sent_padded, 
                'y':self.labels[index], 
                'x_len':len(vectorized_sent)}
    
    def __len__(self):
        return self._len
    
    def vectorize(self, tokens):
        """
        :param tokens: Tokens that should be vectorized. 
        :type tokens: list(str)
        """
        # See https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx 
        # Lets just cast list of indices into torch tensors directly =)
        return torch.tensor(self.vocab.doc2idx(tokens))
    
    def unvectorize(self, indices):
        """
        :param indices: Converts the indices back to tokens.
        :type tokens: list(int)
        """
        return [self.vocab[i] for i in indices]

In [25]:
import gensim
gensim.__version__

'3.8.1'

In [30]:
#! python -m pip install -U pip
#! python -m pip install -U gensim

In [27]:
label_column_names = "toxic	severe_toxic	obscene	threat	insult	identity_hate".split()
toxic_data = ToxicDataset(df_train['comment_text_tokenzied'],
                          df_train[label_column_names].values)

In [31]:
print(toxic_data.texts[123])

['Should', 'say', 'something', 'about', 'his', 'views', 'as', 'an', 'educationalist', 'and', 'socialist', 'political', 'commentator', '.', 'Link', 'to', 'http', ':', '//www.langandlit.ualberta.ca/Fall2004/SteigelBainbridge.html', 'mentions', 'this', 'a', 'bit', '-', 'he', 'stood', 'as', 'an', 'election', 'candidate', 'for', 'Respect', '.']


In [52]:
len(toxic_data)

159571

In [38]:
batch_size = 50
dataloader = DataLoader(dataset=toxic_data, 
                        batch_size=batch_size, shuffle=True)

In [39]:
next(iter(dataloader))

{'x': tensor([[ 6, 15, 38,  ...,  0,  0,  0],
         [50, 44, 51,  ...,  0,  0,  0]]), 'y': tensor([[0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0]]), 'x_len': tensor([49, 27])}

In [18]:
class FFNet(nn.Module):
    def __init__(self, max_len, num_labels, vocab_size, embedding_size, hidden_dim):
        super(FFNet, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size,
                                       embedding_dim=embedding_size, 
                                       padding_idx=0)
        # The no. of inputs to the linear layer is the 
        # no. of tokens in each input * embedding_size
        self.linear1 = nn.Linear(embedding_size*max_len, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, num_labels)
    
    def forward(self, inputs):
        # We want to flatten the inputs so that we get the matrix of shape.
        # batch_size x no. of tokens in each input * embedding_size
        batch_size, max_len = inputs.shape
        embedded = self.embeddings(inputs).view(batch_size, -1)
        hid = F.relu(self.linear1(embedded))
        out = self.linear2(hid)
        return torch.sigmoid(out)
        

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_size = 100
learning_rate = 0.003
hidden_size = 100


criterion = nn.BCELoss()
# Hint: the CBOW model object you've created.
model = FFNet(toxic_data.max_len, 
              len(label_column_names),
              toxic_data.vocab_size, 
              embedding_size=embedding_size, 
              hidden_dim=hidden_size).to(device)


optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#model = nn.DataParallel(model)

training_losses = []
num_epochs = 100
for _e in range(num_epochs):
    epoch_loss = []
    for batch in tqdm(dataloader):
        x = batch['x'].to(device)
        y = batch['y'].to(device)
        # Zero gradient.
        optimizer.zero_grad()
        # Feed forward.
        predictions = model(x)
        loss = criterion(predictions, y.float())
        loss.backward()
        optimizer.step()
        epoch_loss.append(float(loss))
        break
    print(sum(epoch_loss)/len(epoch_loss))
    training_losses.append(sum(epoch_loss)/len(epoch_loss))
    
     

  0%|          | 0/320 [00:00<?, ?it/s]

0.697128176689148


  0%|          | 0/320 [00:00<?, ?it/s]

0.43457943201065063


  0%|          | 0/320 [00:00<?, ?it/s]

...

0.21235232055187225


  0%|          | 0/320 [00:00<?, ?it/s]

0.22905446588993073


  0%|          | 0/320 [00:00<?, ?it/s]

0.18457254767417908


In [20]:

def predict(text):
    # Vectorize and Pad.
    vectorized_sent = toxic_data.vectorize(word_tokenize(text))
    pad_dim = (0, toxic_data.max_len - len(vectorized_sent))
    vectorized_sent = F.pad(vectorized_sent, pad_dim, 'constant')
    # Forward Propagation.
    # Unsqueeze because model is expecting `batch_size` x `sequence_len` shape.
    outputs = model(vectorized_sent.unsqueeze(0)).squeeze()
    # To get the boolean output, we check if outputs are > 0.5
    return [int(l > 0.5) for l in outputs]
    # What happens if you use torch.max instead? =)
    ##return label_column_names[int(torch.max(outputs, dim=1).indices)]

In [21]:
text = "This is a nice message."

In [22]:
print(label_column_names)
predict(text)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


[0, 0, 1, 1, 0, 0]

# Lets do "quick and dirty" with Simple Transformers

In [43]:
! python3 -m pip install -U --user simpletransformers

  Using cached https://files.pythonhosted.org/packages/b3/db/801999c78cf949652d16096fd2d998e0f405c433f9a16983aa2dfbcd3987/simpletransformers-0.18.11-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/42/90/01aa4391f9fb3ec00c1aaa0928b101c0a3a2ec337adac5000aa3c8983e15/wandb-0.8.21-py2.py3-none-any.whl
Processing /Users/liling.tan/Library/Caches/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68/seqeval-0.0.12-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f5/e8/f6bd1eee09314e7e6dee49cbe2c5e22314ccdb38db16c9fc72d2fa80d054/docker_pycreds-0.4.0-py2.py3-none-any.whl
Processing /Users/liling.tan/Library/Caches/pip/wheels/3f/eb/fd/69e5177f67b505e44acbd1aedfbe44b91768ee0c4cd5636576/shortuuid-0.5.0-cp36-none-any.whl
Processing /Users/liling.tan/Library/Caches/pip/wheels/e4/1d/06/640c93f5270d67d0247f30be91f232700d19023f9e66d735c7/nvidia_ml_py3-7.352.0-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/23/5a/f

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [44]:
from simpletransformers.classification import MultiLabelClassificationModel

In [47]:
training_data_for_st = list(zip(toxic_data.texts, toxic_data.labels))

train_df = pd.DataFrame(training_data_for_st, columns=['text', 'labels'])

In [51]:
print(training_data_for_st[0])

(['Explanation', 'Why', 'the', 'edits', 'made', 'under', 'my', 'username', 'Hardcore', 'Metallica', 'Fan', 'were', 'reverted', '?', 'They', 'were', "n't", 'vandalisms', ',', 'just', 'closure', 'on', 'some', 'GAs', 'after', 'I', 'voted', 'at', 'New', 'York', 'Dolls', 'FAC', '.', 'And', 'please', 'do', "n't", 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'I', "'m", 'retired', 'now.89.205.38.27'], tensor([0, 0, 0, 0, 0, 0]))


In [50]:
model = MultiLabelClassificationModel('roberta', 'roberta-base', 
                                      num_labels=6, 
                                      args={'reprocess_input_data': True, 
                                            'overwrite_output_dir': True, 
                                            'num_train_epochs': 5},
                                     use_cuda=False)


In [ ]:
model.train_model(train_df)

Converting to features started. Cache is not used.
